In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dta3.csv']


In [2]:
dt=pd.read_csv('../input/dta3.csv')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dt=pd.get_dummies(dt)

In [4]:
dt=dt.drop(columns=['B_ELECTRONIC_AUCTION_N','PROCEDURE_RES','SECTOR_na','B_GPA_N','TYPE_OF_CONTRACT_W','CAE_TYPE_Z','AUTHORITY_COUNTRY_UK'] )

In [5]:
print(dt.columns)
len(dt)

Index(['YEAR', 'NUMBER_OFFERS', 'AWARD_EST_VALUE_EURO', 'AWARD_VALUE_EURO',
       'AUTHORITY_COUNTRY_AT', 'AUTHORITY_COUNTRY_BE', 'AUTHORITY_COUNTRY_BG',
       'AUTHORITY_COUNTRY_CH', 'AUTHORITY_COUNTRY_CY', 'AUTHORITY_COUNTRY_CZ',
       ...
       'SECTOR_93', 'SECTOR_98', 'B_GPA_Y', 'PROCEDURE_AWP', 'PROCEDURE_COD',
       'PROCEDURE_INP', 'PROCEDURE_NIC', 'PROCEDURE_NOC', 'PROCEDURE_OPE',
       'B_ELECTRONIC_AUCTION_Y'],
      dtype='object', length=145)


2875612

In [6]:
dt = dt[pd.notnull(dt['AWARD_VALUE_EURO'])]
dt = dt[dt['AWARD_VALUE_EURO'] != 0]
len(dt)



2875612

In [7]:
dt = dt[pd.notnull(dt['AWARD_EST_VALUE_EURO'])]
dt = dt[dt['AWARD_EST_VALUE_EURO'] != 0]
len(dt)



1476395

In [8]:
dt = dt[pd.notnull(dt['NUMBER_OFFERS'])]
dt = dt[dt['NUMBER_OFFERS'] != 0]
len(dt)

1433056

In [9]:
import numpy as np
dt['LNRATIO']=(np.log(dt['AWARD_VALUE_EURO']/dt['AWARD_EST_VALUE_EURO']))
len(dt)


1433056

In [10]:
#log of 'df_train['month'] = pd.to_datetime(df_train.Date).dt.month
dt['LN_AWARD_VALUE_EURO'] = np.log(dt['AWARD_VALUE_EURO'])
dt['LN_AWARD_EST_VALUE_EURO'] = np.log(dt['AWARD_EST_VALUE_EURO'])

In [11]:
sd=dt['LNRATIO'].std()
mn=dt['LNRATIO'].mean()
ub=mn+3*sd
lb=mn-3*sd

In [12]:
dt = dt[(dt.LNRATIO>lb) & (dt.LNRATIO<ub)]


In [13]:
train=dt[(dt.YEAR<2017)]
test=dt[(dt.YEAR==2017)]

In [14]:
y_train = train['LNRATIO']
X_train = train.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO', 'LNRATIO', 'LN_AWARD_VALUE_EURO'], axis=1)
y_test = test['LNRATIO']
X_test = test.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO',  'LNRATIO' ,'LN_AWARD_VALUE_EURO'], axis=1)


In [15]:
import math
from scipy.stats import spearmanr
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    print('Train rmse '+str(rmse(m.predict(X_train), y_train))) 
    print('Test  rmse '+str(rmse(m.predict(X_test), y_test)))
    print('Train '+str(spearmanr(m.predict(X_train),y_train)))
    print('Test  '+str(spearmanr(m.predict(X_test),y_test)))

In [16]:
import numpy as np
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
 
from sklearn.datasets import load_boston
from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     GridSearchCV, RandomizedSearchCV)
from sklearn.metrics import r2_score
 
from lightgbm.sklearn import LGBMRegressor

In [17]:
from functools import partial
from hyperopt import fmin, hp, tpe, Trials, space_eval
 
# Define searched space
hyper_space = {'n_estimators': 1000+ hp.randint('n_estimators', 1000),
               'max_depth':  hp.choice('max_depth', [4, 5, 8, -1]),
               'num_leaves': hp.choice('num_leaves', [15, 31, 63, 127]),
               'subsample': hp.uniform('subsample', 0.6, 1.0),
               'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)}

In [18]:
def evaluate(params, X, y):
    
    # Initilize instance of estimator
    est = LGBMRegressor(boosting='gbdt', n_jobs=-1, random_state=2018)
    
    # Set params
    est.set_params(**params)
    
    # Calc CV score
    scores = cross_val_score(estimator=est, X=X, y=y, 
                            scoring='neg_mean_squared_error', cv=4)
    score = np.mean(scores)
 
    return score
 
# Objective minizmied 
hyperopt_objective = lambda params: (-1)*evaluate(params, X_train, y_train)

In [19]:
# Trail
trials = Trials()
 
# Set algoritm parameters
algo = partial(tpe.suggest, 
               n_startup_jobs=20, gamma=0.25, n_EI_candidates=24)
 
# Fit Tree Parzen Estimator
best_vals = fmin(hyperopt_objective, space=hyper_space,
                 algo=algo, max_evals=60, trials=trials,
                 rstate=np.random.RandomState(seed=2018))
 
# Print best parameters
best_params = space_eval(hyper_space, best_vals)
print("BEST PARAMETERS: " + str(best_params))
 
# Print best CV score
scores = [-trial['result']['loss'] for trial in trials.trials]
print("BEST CV SCORE: " + str(np.min(scores)))
 
# Print execution time
tdiff = trials.trials[-1]['book_time'] - trials.trials[0]['book_time']
print("ELAPSED TIME: " + str(tdiff.total_seconds() / 60))

100%|██████████| 60/60 [3:17:21<00:00, 171.83s/it, best loss: 0.23536797479608532]
BEST PARAMETERS: {'colsample_bytree': 0.6154519141597878, 'max_depth': 8, 'n_estimators': 1216, 'num_leaves': 31, 'subsample': 0.7841552053418989}
BEST CV SCORE: -0.23962693369074092
ELAPSED TIME: 194.47948333333335


In [20]:
# Set params

est = LGBMRegressor(**best_params) 
# Fit    
est.fit(X_train, y_train)
y_pred = est.predict(X_test)
 
# Predict
score = rmse(y_test, y_pred)
r2a = r2_score(y_test, y_pred)
print("RMSE SCORE ON TEST DATA: {}".format(score))
print("R2 SCORE ON TEST DATA: {}".format(r2a))

RMSE SCORE ON TEST DATA: 0.4920816778967577
R2 SCORE ON TEST DATA: 0.15368483750128492
